# **Slowly Changing Dimension Builder**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable

In [0]:
# ## Catalog Name
# catalog = "workspace"

# ## Key cols list
# key_cols = "['flight_id']"
# key_cols_list = eval(key_cols)


# ## CDC column
# cdc_col = "modifiedDate"

# ## Backdated Refresh
# backdated_refresh = ""

# ## Source Object
# source_object = "silver_flights"

# ## Source Schema
# source_schema = "silver"

# ## Target Schema
# target_schema = "gold"

# ## Target object
# target_object = "DimFlights"

# ## Surrogate Key
# surrogate_key = "DimFlightsKey"


In [0]:
## Catalog Name
catalog = "workspace"

## Key cols list
key_cols = "['passenger_id']"
key_cols_list = eval(key_cols)


## CDC column
cdc_col = "modifiedDate"

## Backdated Refresh
backdated_refresh = ""

## Source Object
source_object = "silver_customers"

## Source Schema
source_schema = "silver"

## Target Schema
target_schema = "gold"

## Target object
target_object = "DimCustomers"

## Surrogate Key
surrogate_key = "DimCustomersKey"


In [0]:
%sql
SELECT * FROM workspace.silver.silver_customers

passenger_id,name,gender,nationality,modifiedDate
P0001,Kevin Ferguson,Male,Reunion,2025-07-27T11:51:55.456Z
P0002,Kathleen Martinez DVM,Female,Burkina Faso,2025-07-27T11:51:55.456Z
P0003,Cynthia Frazier,Male,Marshall Islands,2025-07-27T11:51:55.456Z
P0004,Ryan Ramsey,Male,Niger,2025-07-27T11:51:55.456Z
P0005,Mike Kim,Male,Taiwan,2025-07-27T11:51:55.456Z
P0006,Diana Adams,Male,Mayotte,2025-07-27T11:51:55.456Z
P0007,Sharon Moon,Male,Madagascar,2025-07-27T11:51:55.456Z
P0008,Cheryl Glenn,Male,Maldives,2025-07-27T11:51:55.456Z
P0009,Allen Lowery,Male,Rwanda,2025-07-27T11:51:55.456Z
P0010,Maria Medina,Male,Denmark,2025-07-27T11:51:55.456Z


In [0]:
# ## Catalog Name
# catalog = "workspace"

# ## Key cols list
# key_cols = "['airport_id']"
# key_cols_list = eval(key_cols)


# ## CDC column
# cdc_col = "modifiedDate"

# ## Backdated Refresh
# backdated_refresh = ""

# ## Source Object
# source_object = "silver_airports"

# ## Source Schema
# source_schema = "silver"

# ## Target Schema
# target_schema = "gold"

# ## Target object
# target_object = "DimAirports"

# ## Surrogate Key
# surrogate_key = "DimAirportsKey"


## **Incremental Data Ingestion**

In [0]:
# No back date refresh
if len(backdated_refresh) == 0:
  # If Table exists, get the last load date
  if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
    last_load = spark.sql(f"SELECT MAX({cdc_col}) FROM {catalog}.{target_schema}.{target_object}")
  else:
    last_load = "1900-01-01 00:00:00"
# If backdated refresh is provided, use that date
else:
  last_load = backdated_refresh

# Test to see if the last load
last_load

'1900-01-01 00:00:00'

In [0]:
last_load

'1900-01-01 00:00:00'

## **Read only Changed Source Data**

In [0]:
df_src = spark.sql(f"SELECT * FROM workspace.{source_schema}.{source_object} WHERE '{cdc_col}' > '{last_load}'")

In [0]:
df_src.display()

passenger_id,name,gender,nationality,modifiedDate
P0001,Kevin Ferguson,Male,Reunion,2025-07-27T11:51:55.456Z
P0002,Kathleen Martinez DVM,Female,Burkina Faso,2025-07-27T11:51:55.456Z
P0003,Cynthia Frazier,Male,Marshall Islands,2025-07-27T11:51:55.456Z
P0004,Ryan Ramsey,Male,Niger,2025-07-27T11:51:55.456Z
P0005,Mike Kim,Male,Taiwan,2025-07-27T11:51:55.456Z
P0006,Diana Adams,Male,Mayotte,2025-07-27T11:51:55.456Z
P0007,Sharon Moon,Male,Madagascar,2025-07-27T11:51:55.456Z
P0008,Cheryl Glenn,Male,Maldives,2025-07-27T11:51:55.456Z
P0009,Allen Lowery,Male,Rwanda,2025-07-27T11:51:55.456Z
P0010,Maria Medina,Male,Denmark,2025-07-27T11:51:55.456Z


## **Read Existing target or simulate empty for first time run**

In [0]:
if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):

    # Key Columns String for Incremental
    key_cols_string_incremental = ", ".join(key_cols_list)

    df_trg = spark.sql(f"""SELECT {key_cols_string_incremental}, {surrogate_key}, create_date, update_date FROM {catalog}.{target_schema}.{target_object}""")

else:
    key_cols_string_init = [f" '' AS {col}" for col in key_cols_list]
    key_cols_string_init = ", ".join(key_cols_string_init)

    df_trg = spark.sql(f"""SELECT {key_cols_string_init}, CAST('0' AS INT) AS {surrogate_key}, CAST('1900-01-01 00:00:00' AS TIMESTAMP) AS create_date, CAST    ('1900-01-01 00:00:00' AS TIMESTAMP) AS update_date WHERE 1 = 0""")

df_trg.display()

passenger_id,DimCustomersKey,create_date,update_date


## **Applying and Peforming Left Join**

In [0]:
join_condition = " AND ".join(f"src.{col} = trg.{col}" for col in key_cols_list)

In [0]:
df_src.createOrReplaceTempView("src")
df_trg.createOrReplaceTempView("trg")

df_join = spark.sql(f"""
                    SELECT
                      src.*,
                      trg.{surrogate_key} AS {surrogate_key},
                      trg.create_date,
                      trg.update_date
                    FROM
                      src
                    LEFT JOIN
                      trg
                    ON
                      {join_condition}""")

In [0]:
df_join.display()

passenger_id,name,gender,nationality,modifiedDate,DimCustomersKey,create_date,update_date
P0001,Kevin Ferguson,Male,Reunion,2025-07-27T11:51:55.456Z,null,null,null
P0002,Kathleen Martinez DVM,Female,Burkina Faso,2025-07-27T11:51:55.456Z,null,null,null
P0003,Cynthia Frazier,Male,Marshall Islands,2025-07-27T11:51:55.456Z,null,null,null
P0004,Ryan Ramsey,Male,Niger,2025-07-27T11:51:55.456Z,null,null,null
P0005,Mike Kim,Male,Taiwan,2025-07-27T11:51:55.456Z,null,null,null
P0006,Diana Adams,Male,Mayotte,2025-07-27T11:51:55.456Z,null,null,null
P0007,Sharon Moon,Male,Madagascar,2025-07-27T11:51:55.456Z,null,null,null
P0008,Cheryl Glenn,Male,Maldives,2025-07-27T11:51:55.456Z,null,null,null
P0009,Allen Lowery,Male,Rwanda,2025-07-27T11:51:55.456Z,null,null,null
P0010,Maria Medina,Male,Denmark,2025-07-27T11:51:55.456Z,null,null,null


**Split into old records and new records**

In [0]:
df_old_records = df_join.filter(col(f"{surrogate_key}").isNotNull())
df_new_records = df_join.filter(col(f"{surrogate_key}").isNull())


In [0]:
df_old_records.display()
df_new_records.display()

passenger_id,name,gender,nationality,modifiedDate,DimCustomersKey,create_date,update_date


passenger_id,name,gender,nationality,modifiedDate,DimCustomersKey,create_date,update_date
P0001,Kevin Ferguson,Male,Reunion,2025-07-27T11:51:55.456Z,null,null,null
P0002,Kathleen Martinez DVM,Female,Burkina Faso,2025-07-27T11:51:55.456Z,null,null,null
P0003,Cynthia Frazier,Male,Marshall Islands,2025-07-27T11:51:55.456Z,null,null,null
P0004,Ryan Ramsey,Male,Niger,2025-07-27T11:51:55.456Z,null,null,null
P0005,Mike Kim,Male,Taiwan,2025-07-27T11:51:55.456Z,null,null,null
P0006,Diana Adams,Male,Mayotte,2025-07-27T11:51:55.456Z,null,null,null
P0007,Sharon Moon,Male,Madagascar,2025-07-27T11:51:55.456Z,null,null,null
P0008,Cheryl Glenn,Male,Maldives,2025-07-27T11:51:55.456Z,null,null,null
P0009,Allen Lowery,Male,Rwanda,2025-07-27T11:51:55.456Z,null,null,null
P0010,Maria Medina,Male,Denmark,2025-07-27T11:51:55.456Z,null,null,null


## **Updating old records with new update timestamp**

In [0]:
df_old_enr = df_old_records.withColumn('update_date', current_timestamp())
df_old_enr.display()

passenger_id,name,gender,nationality,modifiedDate,DimCustomersKey,create_date,update_date


## **Assigning Surrogate key and timestamps to new records**

In [0]:
if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
  max_surrogate_key = spark.sql(f"""
                                SELECT 
                                  MAX({surrogate_key})
                                FROM  
                                  {catalog}.{target_schema}.{target_object}
                                """).collect()[0][0]
  df_new_enr = df_new_records.withColumn(f"{surrogate_key}", lit(max_surrogate_key) + lit(1) + monotonically_increasing_id())\
                             .withColumn('create_date', current_timestamp())\
                             .withColumn('update_date', current_timestamp())

else:
  max_surrogate_key = 0  
  df_new_enr = df_new_records.withColumn(f"{surrogate_key}", lit(max_surrogate_key) + lit(1) + monotonically_increasing_id())\
                             .withColumn('create_date', current_timestamp())\
                             .withColumn('update_date', current_timestamp())
  

In [0]:
df_new_enr.display()

passenger_id,name,gender,nationality,modifiedDate,DimCustomersKey,create_date,update_date
P0001,Kevin Ferguson,Male,Reunion,2025-07-27T11:51:55.456Z,1,2025-07-28T18:22:43.613Z,2025-07-28T18:22:43.613Z
P0002,Kathleen Martinez DVM,Female,Burkina Faso,2025-07-27T11:51:55.456Z,2,2025-07-28T18:22:43.613Z,2025-07-28T18:22:43.613Z
P0003,Cynthia Frazier,Male,Marshall Islands,2025-07-27T11:51:55.456Z,3,2025-07-28T18:22:43.613Z,2025-07-28T18:22:43.613Z
P0004,Ryan Ramsey,Male,Niger,2025-07-27T11:51:55.456Z,4,2025-07-28T18:22:43.613Z,2025-07-28T18:22:43.613Z
P0005,Mike Kim,Male,Taiwan,2025-07-27T11:51:55.456Z,5,2025-07-28T18:22:43.613Z,2025-07-28T18:22:43.613Z
P0006,Diana Adams,Male,Mayotte,2025-07-27T11:51:55.456Z,6,2025-07-28T18:22:43.613Z,2025-07-28T18:22:43.613Z
P0007,Sharon Moon,Male,Madagascar,2025-07-27T11:51:55.456Z,7,2025-07-28T18:22:43.613Z,2025-07-28T18:22:43.613Z
P0008,Cheryl Glenn,Male,Maldives,2025-07-27T11:51:55.456Z,8,2025-07-28T18:22:43.613Z,2025-07-28T18:22:43.613Z
P0009,Allen Lowery,Male,Rwanda,2025-07-27T11:51:55.456Z,9,2025-07-28T18:22:43.613Z,2025-07-28T18:22:43.613Z
P0010,Maria Medina,Male,Denmark,2025-07-27T11:51:55.456Z,10,2025-07-28T18:22:43.613Z,2025-07-28T18:22:43.613Z


## **Union Old destination records with new and merge to delta table**

In [0]:
df_union = df_old_enr.union(df_new_enr)

In [0]:
display(df_union)

passenger_id,name,gender,nationality,modifiedDate,DimCustomersKey,create_date,update_date
P0001,Kevin Ferguson,Male,Reunion,2025-07-27T11:51:55.456Z,1,2025-07-28T18:22:45.820Z,2025-07-28T18:22:45.820Z
P0002,Kathleen Martinez DVM,Female,Burkina Faso,2025-07-27T11:51:55.456Z,2,2025-07-28T18:22:45.820Z,2025-07-28T18:22:45.820Z
P0003,Cynthia Frazier,Male,Marshall Islands,2025-07-27T11:51:55.456Z,3,2025-07-28T18:22:45.820Z,2025-07-28T18:22:45.820Z
P0004,Ryan Ramsey,Male,Niger,2025-07-27T11:51:55.456Z,4,2025-07-28T18:22:45.820Z,2025-07-28T18:22:45.820Z
P0005,Mike Kim,Male,Taiwan,2025-07-27T11:51:55.456Z,5,2025-07-28T18:22:45.820Z,2025-07-28T18:22:45.820Z
P0006,Diana Adams,Male,Mayotte,2025-07-27T11:51:55.456Z,6,2025-07-28T18:22:45.820Z,2025-07-28T18:22:45.820Z
P0007,Sharon Moon,Male,Madagascar,2025-07-27T11:51:55.456Z,7,2025-07-28T18:22:45.820Z,2025-07-28T18:22:45.820Z
P0008,Cheryl Glenn,Male,Maldives,2025-07-27T11:51:55.456Z,8,2025-07-28T18:22:45.820Z,2025-07-28T18:22:45.820Z
P0009,Allen Lowery,Male,Rwanda,2025-07-27T11:51:55.456Z,9,2025-07-28T18:22:45.820Z,2025-07-28T18:22:45.820Z
P0010,Maria Medina,Male,Denmark,2025-07-27T11:51:55.456Z,10,2025-07-28T18:22:45.820Z,2025-07-28T18:22:45.820Z


In [0]:
if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
    dlt_obj = DeltaTable.forName(spark, f"{catalog}.{target_schema}.{target_object}")
    dlt_obj.alias("trg").merge(df_union.alias("src"), f"trg.{surrogate_key} = src.{surrogate_key}")\
                        .whenMatchedUpdateAll(condition = f"src.{cdc_col} >= trg.{cdc_col}")\
                        .whenNotMatchedInsertAll()\
                        .execute()


else:
    df_union.write.format("delta")\
            .mode("append")\
            .saveAsTable(f"{catalog}.{target_schema}.{target_object}")

In [0]:
%sql
SELECT * FROM workspace.gold.dimflights

flight_id,airline,origin,destination,flight_date,modifiedDate,DimFlightsKey,create_date,update_date
F0001,Delta,Kellyfort,South Kathleen,2025-05-04,2025-07-27T11:51:55.440Z,1,2025-07-28T18:21:46.931Z,2025-07-28T18:21:46.931Z
F0002,Qatar Airways,Lake Stephen,New Vincent,2025-04-29,2025-07-27T11:51:55.440Z,2,2025-07-28T18:21:46.931Z,2025-07-28T18:21:46.931Z
F0004,Delta,Maddenshire,Johnchester,2025-05-16,2025-07-27T11:51:55.440Z,3,2025-07-28T18:21:46.931Z,2025-07-28T18:21:46.931Z
F0006,Air Canada,New Richardside,South Jamesborough,2025-05-16,2025-07-27T11:51:55.440Z,4,2025-07-28T18:21:46.931Z,2025-07-28T18:21:46.931Z
F0007,Delta,Berryport,Miguelburgh,2025-05-24,2025-07-27T11:51:55.440Z,5,2025-07-28T18:21:46.931Z,2025-07-28T18:21:46.931Z
F0008,Lufthansa,Briannachester,Cervantesland,2025-05-26,2025-07-27T11:51:55.440Z,6,2025-07-28T18:21:46.931Z,2025-07-28T18:21:46.931Z
F0009,Delta,Alexandraborough,North Alexishaven,2025-06-10,2025-07-27T11:51:55.440Z,7,2025-07-28T18:21:46.931Z,2025-07-28T18:21:46.931Z
F0010,Emirates,Kruegerchester,Martintown,2025-05-20,2025-07-27T11:51:55.440Z,8,2025-07-28T18:21:46.931Z,2025-07-28T18:21:46.931Z
F0011,Lufthansa,Port Joannahaven,North Jasonton,2025-05-30,2025-07-27T11:51:55.440Z,9,2025-07-28T18:21:46.931Z,2025-07-28T18:21:46.931Z
F0012,Air Canada,New Hunter,East Maria,2025-04-23,2025-07-27T11:51:55.440Z,10,2025-07-28T18:21:46.931Z,2025-07-28T18:21:46.931Z


In [0]:
%sql
SELECT * FROM workspace.gold.dimflights WHERE flight_id = 'F0005'

flight_id,airline,origin,destination,flight_date,modifiedDate,DimFlightsKey,create_date,update_date
F0005,IndiGo,Bennettside,New Mistyhaven,2025-06-24,2025-07-27T21:34:27.168Z,111,2025-07-28T18:21:46.931Z,2025-07-28T18:21:46.931Z
